In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from datetime import datetime
import shap
import json
import joblib
from pycaret.classification import *

In [2]:
path = 'c:\\Users\\wongi\\Desktop\\알고리즘랩스\\프로젝트\\현대산업개발\\develop'

In [3]:
raw_data = pd.read_csv(f'{path}\\algorithmlabs.inference\\inference_data_attrition.csv')
model = joblib.load(f'{path}\\algorithmlabs.inference\\pickle_attrition\\model.pkl')
explainer = joblib.load(f'{path}\\algorithmlabs.inference\\pickle_attrition\\explainer.pkl')
encoder_1 = joblib.load(f'{path}\\algorithmlabs.inference\\pickle_attrition\\label_encoder.pkl')
encoder_2 = joblib.load(f'{path}\\algorithmlabs.inference\\pickle_attrition\\label_encoder_2.pkl')
name_dict = joblib.load(f'{path}\\algorithmlabs.inference\\pickle_attrition\\name_dict.pkl')

In [4]:
inf_data = raw_data.copy()

In [5]:
inf_data['YN_HANDICAP'].replace({'Y' : 1, 'N' : 0}, inplace=True)
inf_data['YN_SUPPORTED'].replace({'Y' : 1, 'N' : 0}, inplace=True)
inf_data['YN_FOREIGNER'].replace({'Y' : 1, 'N' : 0}, inplace=True)
inf_data['YN_GENDER'].replace({'M' : 1, 'W' : 0}, inplace=True)

In [6]:
for label in np.unique(inf_data['DS_BIRTHPLACE']):
    if label not in encoder_1.classes_: 
        encoder_1.classes_ = np.append(encoder_1.classes_, label) 
inf_data['DS_BIRTHPLACE'] = encoder_1.transform(inf_data['DS_BIRTHPLACE'])

In [7]:
for label in np.unique(inf_data['CD_OCCUPATION']):
    if label not in encoder_1.classes_: 
        encoder_1.classes_ = np.append(encoder_1.classes_, label) 
inf_data['CD_OCCUPATION'] = encoder_1.transform(inf_data['CD_OCCUPATION'])

In [8]:
inf_data.drop(["ID_SABUN", "DS_JOBTYPE", "DS_POSITION", "DS_DEPT", "DS_BONBU", "DS_TY_DEPT", "DS_OCCUPATION", "DS_DUTY", 
               "DS_ADOPTYPE", "DS_JOBFAMILY", "DS_JOBFAMILY_JOIN", 'DS_HNAME', 'DS_HANDPHONE', 'DS_EMAIL'], axis=1, inplace=True)

In [9]:
predict_proba = model.predict_proba(inf_data)[:,1]
raw_data['predict_proba'] = predict_proba

In [10]:
raw_data = raw_data[raw_data.columns[0:1].tolist() + raw_data.columns[-1:].tolist() + raw_data.columns[1:-1].tolist()]

In [11]:
columns = ['ID_SABUN', 'predict_proba', 'DS_HNAME', 'DS_POSITION', 'DS_OCCUPATION', 'DS_HANDPHONE', 'DS_EMAIL', 
           'DS_BONBU', 'DS_DEPT', 'DS_JOBFAMILY', 'DT_BIRTH', 'DS_BIRTHPLACE', 'DT_GROUPJOIN', 'DT_JOIN', 'CD_JOBTYPE',
       'CD_POSITION', 'CD_DEPT', 'CD_BONBU', 'TY_DEPT', 'CD_OCCUPATION',
       'CD_DUTY', 'CD_ADOPTYPE', 'YN_HANDICAP', 'YN_SUPPORTED', 'YN_FOREIGNER', 'YN_GENDER', 'CD_JOBFAMILY',
       'CD_JOBFAMILY_JOIN', 'DS_JOBTYPE', 'DS_TY_DEPT', 'DS_DUTY', 'DS_ADOPTYPE', 'DS_JOBFAMILY_JOIN', 
       'VAT_THREE_MONTH_AGO', 'VAT_TWO_MONTH_AGO', 'VAT_ONE_MONTH_AGO', 'FINAL_GRADE_RECENT_YEAR', 'WEAK_ONE', 'WEAK_TWO', 'WEAK_THREE',
       'WEAK_FOUR', 'WEAK_FIVE', 'WEAK_SIX', 'EDU_CNT']
raw_data = raw_data[columns]

In [12]:
raw_data.drop(["CD_JOBTYPE", "CD_POSITION", "CD_DEPT", "CD_BONBU", "TY_DEPT", "CD_OCCUPATION", "CD_DUTY", "CD_ADOPTYPE", "CD_JOBFAMILY", "CD_JOBFAMILY_JOIN"], axis=1, inplace=True)

In [13]:
shap_values = explainer.shap_values(inf_data, check_additivity=False)
new_shap=[]

if len(shap_values) == 2 :
    for i in range(len(shap_values[1])) :
        new_shap.append(shap_values[1][i]/abs(shap_values[1][i]).sum())
else : 
    for i in range(len(shap_values)) :
        new_shap.append(shap_values[i]/abs(shap_values[i]).sum())
        
feature_weight = pd.DataFrame(new_shap, columns=inf_data.columns)
feature_weight['ID_SABUN'] = raw_data['ID_SABUN']
feature_weight = feature_weight[feature_weight.columns[-1:].tolist() + feature_weight.columns[:-1].tolist()]

In [14]:
for name in raw_data.columns.tolist() : 
    for name_ex in name_dict.keys() : 
        if name == name_ex : 
            raw_data.rename(columns={name : name_dict[name_ex]}, inplace=True)

In [15]:
for name in feature_weight.columns.tolist() : 
    for name_ex in name_dict.keys() : 
        if name == name_ex : 
            feature_weight.rename(columns={name : name_dict[name_ex]}, inplace=True)

In [16]:
raw_data.to_csv(f'{path}\\algorithmlabs.inference\\infer_result_attrition.csv', index=False)
feature_weight.to_csv(f'{path}\\algorithmlabs.inference\\feature_weight_attrition.csv', index=False)